<a href="https://colab.research.google.com/github/mitreilab/autonomousfuture/blob/master/1210reilAutomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import OES Data

In [0]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as pp
!wget "https://www.dropbox.com/s/t4444pduqc7rtgs/recentyears.csv?dl=0" -q -O "recent.csv" 
!wget "https://www.dropbox.com/s/6gglwj5jedmfvhi/oldyears.csv?dl=0" -q -O "old.csv" 
#Download raw csv file from Dropbox and rename to "rawdatanaics.csv"
new = pd.read_csv("recent.csv",low_memory=False)
old=pd.read_csv("old.csv",low_memory=False)


## Import ONet Data ##

In [2]:
!wget "https://www.dropbox.com/s/5cjlzvxqtoq5mm4/combined.sqlite?dl=0" -q -O  "onet.sqlite"  
#Download raw SQLite file from Dropbox and rename to "onet.sqlite"
import sqlite3
onetConnection = sqlite3.connect('onet.sqlite')
cur = onetConnection.cursor()
query = "SELECT * FROM occupation_data";
occdata = pd.read_sql_query(query, onetConnection)
len(occdata)
# Onet surveyed 1109 occupations.


1109

In [0]:
#Download the CSV files that we need for Autor Method
!wget "https://www.dropbox.com/s/pnvz35bcthloy6n/Abilities.csv?dl=0" -q -O "Abilities.csv" 
Abilities=pd.read_csv("Abilities.csv",low_memory=False)
!wget "https://www.dropbox.com/s/5btim1otk24j05d/Skills.csv?dl=0" -q -O "Skills.csv" 
Skills=pd.read_csv("Skills.csv",low_memory=False)
!wget "https://www.dropbox.com/s/9ajd56nnhlrfo2m/Task%20Statements.csv?dl=0" -q -O "Task Statements.csv" 
Tasks = pd.read_csv("Task Statements.csv",low_memory=False)
!wget "https://www.dropbox.com/s/9ez75cfcojjpyur/Work%20Activities.csv?dl=0" -q -O "Work Activities.csv" 
Activities = pd.read_csv("Work Activities.csv",low_memory=False)


## OES Data cleaning

##Link old occ code with occ code after 2010


In [0]:
!wget "https://www.dropbox.com/s/30fryvdkh5qnk1z/crosswalktable.csv?dl=0" -q -O "crosswalk.csv" 
match=pd.read_csv("crosswalk.csv",low_memory=False)


In [9]:
old.columns

Index(['year', 'naics', 'naics_title', 'occ_code', 'occ_title', 'group',
       'tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean',
       'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
       'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90',
       'naics_codedescription', 'occ_code-title', 'detailed'],
      dtype='object')

In [10]:
# Build a dictionary for old code to be transformed into new
oldlist=match['2000 SOC code'].tolist()
newlist=match['2010 SOC code'].tolist()
keys = oldlist
values = newlist
matchbook= dict(zip(keys, values))
matchbook[113041]

113111

In [11]:
tobecorrect=old['occ_code'].tolist()
len(tobecorrect)

7166

In [0]:
for i in range(len(tobecorrect)):
  oldcode=tobecorrect[i]
  try:
    temp=matchbook[oldcode]
    tobecorrect[i]=temp
  except:
    continue
correctcode= tobecorrect   
  


In [0]:

correctcode=pd.DataFrame(correctcode)


In [0]:
old['occ_code']=correctcode

In [0]:
old.reset_index(drop=True, inplace=True)
new.reset_index(drop=True, inplace=True)
result = pd.concat([old,new])

In [17]:
old.shape

(7166, 26)

In [18]:
new.shape

(15628, 26)

In [0]:
result.reset_index(drop=True, inplace=True)

In [20]:
len(newlist)
# There are 792  detailed ocupations in total in US

792

In [23]:
df=result[result['occ_code'].isin(newlist)]
df.shape
# Now we finishe building the OLS dabatase for the 13 Real Estate NAICS



(21264, 26)

In [24]:
# After we choose 13 industries (using 4_digit NAISC)
# We have 528 detailed occupations
targetlist=df['occ_code'].unique()
len(targetlist)

528

In [0]:
df.head()

### Target the occupations that are 1. In Real Estate industry 2. Not General Group 3. Onet has surveyed


In [0]:
# !wget "https://www.dropbox.com/s/329edv8yrn3ik25/Occupation%20Data.csv?dl=0"-q -O  "Occupation Data.csv" 
# occ_onet=pd.read_csv("Occupation Data.csv",low_memory=False)
# #Download 'Occupation Data' CSV that changes the 8-digit Onet-code to 6-digit, ommiting the last two digits.
# del_occ_endwith_00_and_in_Onet=oes.loc[oes['occ_code'].isin(occ_onet['O*NET-SOC Code'])]
# #Delete the occupation if the code is not in 'Occupation Data'
# del_occ_endwith_00_and_in_Onet.shape

In [0]:
# target_occ=set(del_occ_endwith_00_and_in_Onet['occ_code'])
# target_occ=list(target_occ)
# target=pd.DataFrame({'target_occ': target_occ})
# len(target)
# # we target 558 occupations.

### Drop the columns in the oes.csv that contains duplicated or redundant data.

In [0]:
# del_occ_endwith_00_and_in_Onet.columns
# #drop the colomuns that contain duplicate information
# del_duplicated_column=del_occ_endwith_00_and_in_Onet.drop(['title_code','occ_title_code', 'occ_title_mid', 'year.1', 'naics.1', 'naics_title.1', 'occ_code.1', 'occ_title.1', 'group', 'naics_codedescription', 'year-naics-occ_code', 'year-naics-occ_code.1','year.2', 'naics.2', 'naics_title.2', 'occ_code.2', 'title_code.1', 'dominant_lifecycle.1', 'occ_title.2', 'occ_title_code.1', 'occ_title_mid.1','group.1', 'naics_codedescription.1', 'keepsy'],axis=1)
# # the columns # decrease to 37 from 61
# del_duplicated_column.shape

## Call BLS data by 6-digit occupation code

In [0]:
# try to extract data by occ_code 
# dx = del_duplicated_column[del_duplicated_column['occ_code'] == '11-1011']
# dx[['year','naics_title', 'tot_emp_num']].head(3)

In [0]:
# assign the matrix of "year" and "naics_code" to a specific occ_code, call it 'labour_economy_data'
# first, create a function that generate the lb_econ_matrix for an occ_code
def lb_econ (occ_code,field,data):
    # this function yield a data frame for a given occ_code and a field
    # get all unique naics titles in the data
    feature_list = data['naics'].unique()
    # get all unique year in the data
    years = sorted(data['year'].unique())    
    mt_result = pd.DataFrame(0.00, index = years, columns = feature_list)   
    # filter out all rows and columns that have the given occ_code and fields
    df_occ = data[data['occ_code'] == occ_code][['year','naics_title', field]]
    for index, row in df_occ.iterrows():
        try:
            mt_result.loc[row['year']][row['naics_title']] = float(row[field])
        except:
            mt_result.loc[row['year']][row['naics_title']] = np.nan
    return mt_result.round(2)


In [0]:
# An example of how to use the function
# data1=del_duplicated_column
# # the lb_Econ_features contains all the fields' name that we can choose from, there're 17 all together.
# lb_econ_features=['occ_code','tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90']
# #try to input a field name called 'tot_emp_num'
# aa=lb_econ (111011,'tot_emp_num',df)
# aa.head()

In [29]:
df.columns

Index(['year', 'naics', 'naics_title', 'occ_code', 'occ_title', 'group',
       'tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean',
       'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
       'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90',
       'naics_codedescription', 'occ_code-title', 'detailed'],
      dtype='object')

# Object Oriented Programming (OOP)

In [0]:
# laborfeature include:'tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean',
#        'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
#        'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90',
#        'naics_codedescription'
# year include: 2007 to 2016
# industry include:
# 'Residential Building Construction',
# 'Nonresidential Building Construction',
# 'Land Subdivision',
# 'Foundation, Structure, and Building Exterior Contractors',
# 'Building Equipment Contractors',
# 'Building Finishing Contractors',
# 'Other Specialty Trade Contractors',
# 'Lessors of Real Estate',
# 'Offices of Real Estate Agents and Brokers',
# 'Activities Related to Real Estate',
# 'Architectural, Engineering, and Related Services',
# 'Specialized Design Services',
# 'Services to Buildings and Dwellings',
      
class occ(object):
  # add attributes to occ
  def __init__(self,code,title,year,industry,industrycode,name,negotiation,math,GDM,finger,manual):
    self.code=code
    self.title=title
    self.year=year  
    self.industry=industry
    self.industrycode=industrycode
    self.name=name
    self.negotiation=negotiation
    self.math=math
    self.GDM=GDM
    self.finger=finger
    self.manual=manual

#     yearemp,
#               emp_prse, pct_total, pct_rpt, h_mean, a_mean,
#        mean_prse, h_pct10, h_pct25, h_median, h_pct75, h_pct90,
#        a_pct10, a_pct25, a_median_num, a_pct75, a_pct90,
#       naics_codedescription,tasks,abilities,knowledge,skills,work_activities
#     self.yearemp=yearemp
#     self.emp_prse=emp_prse
#     self.pct_total=pct_total
#     self.pct_rpt=pct_rpt
#     self.h_mean=h_mean
#     self.a_mean=a.mean
#     self.mean_prse=mean_prse
#     self.h_pct10=h_pct10
#     self.h_pct25=h_pct25
#     self.h_median=h_median
#     self.h_pct75=h_pct75
#     self.h_pct90=h_pct90
#     self.a_pct10=a_pct10
#     self.a_pct25=a_pct25
#     self.a_median_num=a_median_num
#     self.a_pct75=a_pct75
#     self.a_pct90=a_pct90
#     self.naics_codedescription=naics_codedescription
#     self.laborfeature=laborfeature
#     self.task=tasks
#     self.abilities=abilities
#     self.knowledge=knowledge
#     self.skills=skills
#     self.work_activities=work_activities
  def __str__(self):
    return str(self.code)+str(self.title)+str(self.year)+str(self.industry+str(self.industrycode))
#   def__add__(self,other):
#     result=self.yearemp+other.yearemp
#     return result
#   def__eq__(self,other):
    
  # define methods:
  # total employment calculate employment number of each occ in all 13 industries
  # inputs for this function should be the 13 occ objects
  # with same code,title,year
  # with different industry,industry code,13 industries in the same year
#   def sumemp(self,other):
#     if self.code==other.code and self.year==other.year:
#       return self.yearemp+other.yearemp



## make all the objects for Real Estate using 13 NAICS (528 times 10 years times 13 industries)


In [31]:
row=df.iloc[0]
type(row[['year','naics']])


pandas.core.series.Series

In [0]:
# a=occ(row['occ_code'],row['occ_title'],row['year'],row['naics_title'],row['naics'],'name1')
# b=occ(row2['occ_code'],row2['occ_title'],row2['year'],row2['naics_title'],row2['naics'],'name2')

In [0]:
# newlist.append(a.name)
# newlist

In [0]:
# there are at most 68640 occ objects in total
# now we create them out and name them by 'occ+their occ_code'
listofocc=[]
all_occ=[]
for index, row in df.iterrows():
  #create a new object
  name=str(row['occ_code'])+str(row['occ_title'])+'in year'+str(row['year'])+'in industry'+str(row['naics'])
  temp=occ(row['occ_code'],row['occ_title'],row['year'],row['naics_title'],row['naics'],name)
  listofocc.append(temp.name)
  all_occ.append(temp)
  
  

In [0]:
    # get all unique naics titles in the data
    feature_list = data['naics'].unique()
    # get all unique year in the data
    years = sorted(data['year'].unique())    
    mt_result = pd.DataFrame(0.00, index = years, columns = feature_list)   
    # filter out all rows and columns that have the given occ_code and fields
    df_occ = data[data['occ_code'] == occ_code][['year','naics_title', field]]
    for index, row in df_occ.iterrows():
        try:
            mt_result.loc[row['year']][row['naics_title']] = float(row[field])
        except:
            mt_result.loc[row['year']][row['naics_title']] = np.nan
    return mt_result.round(2)

In [0]:
# # Make a list for 2007, negotiation
# negotiationlist=[]
# for year in range(2007,2017):
#   tempsum=0
#   for occ in all_occ:
#     if occ.year==year:
#       employment=manual[occ]['tot_emp_num']
#       temp1=Skills.loc[Skills['SOC Code']==occ.code]  
#       negotiation_score=temp1.loc[temp1['Element ID']=='2.B.1.d']['Data Value']


In [0]:
occcode=111011
cutocc=cutyear.loc[cutyear['occ_code']==occcode]
employment=cutocc['tot_emp_num'].sum()
temp1=Skills.loc[Skills['SOC Code']==occcode]  
score_series=temp1.loc[temp1['Element ID']=='2.B.1.d']['Data Value']

In [89]:
float(score_series)

4.62

In [0]:
tempsum=0
cutyear=df.loc[df['year']==2007]
occcode=111011
cutocc=cutyear.loc[cutyear['occ_code']==occcode]
employment=cutocc['tot_emp_num'].sum()
temp1=Skills.loc[Skills['SOC Code']==occcode]  
score=float(temp1.loc[temp1['Element ID']=='2.B.1.d']['Data Value']
weightedscore=employment*score


In [77]:
type(weightedscore)

pandas.core.series.Series

In [83]:
float(score)

4.62

In [71]:
len(negotiationlist)

10

In [95]:
negotiationlist

[21923192,
 21643695,
 19831040,
 18090637,
 18014899,
 18335014,
 18767207,
 19461503,
 20162399,
 20784521]

In [0]:
temp1=Skills.loc[Skills['SOC Code']==111011]


In [61]:
temp1.loc[temp1['Element ID']=='2.B.1.d']['Data Value']

13    4.62
Name: Data Value, dtype: float64

In [0]:
 Skills.loc[Skills['SOC Code']=='111011' and ['Element ID']=='2.B.1.d']

In [0]:
for occ in all_occ:
  temp=Skills[Skills['SOC Code']==occ.code and Skills['Element ID']=='2.B.1.d']
  occ.negotiation=

for index,row in Abilities.iterrows():
  

In [34]:
# the list containing all the objects names is named as listofocc
# the list containing all the objects is named as all_occ
len(listofocc)

21264

In [36]:
len(all_occ)

21264

In [0]:
all_occ[2].year

2007

In [0]:
listofocc

## Build the dictionary based on 13 NAISCS of Real Estate

In [0]:
# Build a dictionary where
#   -  keys: the occ object
# Values : all the labor econ attributes and The O_net attributes


keys = all_occ

values =[]
# # We first define some functions that we need to search values from Onet
# # The function to search for tasks for an occpation:
# def gettask(occ_code,Tasks):
#   for index, row in Tasks.iterrows():
#     temp=Tasks[Tasks['occ_code']==occ_code][['Task ID','Task']]
#     onetvalue=temp.values.tolist()
#   return onetvalue

for index, row in df.iterrows():
  #create a sublist to be one element of the values list
  temp=row[['occ_code','tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90']]

  
  values.append(temp)
  
manual= dict(zip(keys, values))

In [62]:
manual[all_occ[1]]['tot_emp_num']


16680.0

In [0]:
# # Then we have a tasklist for each occupation, but this method is slow
# # The function to search for tasks for an occpation:


# def gettask(occ_code,Tasks):
#   for index, row in Tasks.iterrows():
#     temp=Tasks[Tasks['occ_code']==occ_code][['Task ID','Task']]
#   return temp
# tasklist=gettask(111011,Tasks)

In [0]:
# tasklist[2]

[8825,
 'Analyze operations to evaluate performance of a company or its staff in meeting objectives or to determine areas of potential cost reduction, program improvement, or policy change.']

### 2 by 2 metrics to assess 13 NAISCS



In [44]:
# Among 13 NAISCS industries, we have 528 occupations, the list of occupations is named as targetlist
len(targetlist)

528

In [0]:
#   # built a occpationcode list to change 6digit to 8 digit and change the form
# onetform=[]
# for item in targetlist:
#   string=str(item)
#   temp=str(string[:1]) + '-' +str(string[2:])+'.00'
#   onetform.append(temp)
# onetform


#### Make a list for Non-routine cognitive- Interactive

In [0]:
# Make a list for 10 years, use 'negotiation score'in Skills, Element id is 2.B.1.d
negotiationlist=[]
totalemployment=[]
for year in range(2007,2017):
  tempsum=0
  denom=0
  cutyear=df.loc[df['year']==year]
  for occcode in targetlist:
    cutocc=cutyear.loc[cutyear['occ_code']==occcode]
    employment=cutocc['tot_emp_num'].sum()
    denom+=employment
    temp1=Skills.loc[Skills['SOC Code']==occcode]  
    score_series=temp1.loc[temp1['Element ID']=='2.B.1.d']['Data Value']
    try:
      score=float(score_series)
    except:
      continue      
    weightedscore=int(employment*score)
    tempsum+=weightedscore
  valueofyear=tempsum/denom
  negotiationlist.append(valueofyear)
  totalemployment.append(denom)
    

In [115]:
totalemployment

[11420590.0,
 11196570.0,
 10161260.0,
 9177840.0,
 9087980.0,
 9241660.0,
 9500890.0,
 9914140.0,
 10302570.0,
 10656980.0]

In [113]:
negotiationlist

[1.9196199145578294,
 1.9330647689426315,
 1.9516319826478212,
 1.9711214185472834,
 1.9822775798362233,
 1.983952450100956,
 1.9753104182871288,
 1.9630046579935325,
 1.9570261594922431,
 1.950319978080094]

#### Make a list for Non-routine cognitive- Analytic

In [0]:
# Make a list for 10 years, use 'Mathematical Reasoning' in Abilities, Element id is 1.A.1.c.1
MATHlist=[]
for year in range(2007,2017):
  tempsum=0
  cutyear=df.loc[df['year']==year]
  for occcode in targetlist:
    cutocc=cutyear.loc[cutyear['occ_code']==occcode]
    employment=cutocc['tot_emp_num'].sum()
    temp1=Abilities.loc[Abilities['SOC Code']==occcode]  
    score_series=temp1.loc[temp1['Element ID']=='1.A.1.c.1']['Data Value']
    try:
      score=float(score_series)
    except:
      continue      
    weightedscore=int(employment*score)
    tempsum+=weightedscore
  MATHlist.append(tempsum)

In [97]:
MATHlist

[19912594,
 19665161,
 18041007,
 16346415,
 16223788,
 16444917,
 16858215,
 17496147,
 18181594,
 18739184]

#### Make a list for Non-routine Manual

In [0]:
# Make a list for 10 years, use 'Manual Dexterity' in Abilities, Element id is 1.A.2.a.2
Manulist=[]
for year in range(2007,2017):
  tempsum=0
  cutyear=df.loc[df['year']==year]
  for occcode in targetlist:
    cutocc=cutyear.loc[cutyear['occ_code']==occcode]
    employment=cutocc['tot_emp_num'].sum()
    temp1=Abilities.loc[Abilities['SOC Code']==occcode]  
    score_series=temp1.loc[temp1['Element ID']=='1.A.2.a.2']['Data Value']
    try:
      score=float(score_series)
    except:
      continue      
    weightedscore=int(employment*score)
    tempsum+=weightedscore
  Manulist.append(tempsum)

In [108]:
Manulist

[22312815,
 21779841,
 19673674,
 17771392,
 17570343,
 17866598,
 18349511,
 19184264,
 19971208,
 20634037]

#### Make a list for Routine Manual

In [0]:
# Make a list for 10 years, use 'Finger Dexterity' in Abilities, Element id is 1.A.2.a.3
FingDexlist=[]
for year in range(2007,2017):
  tempsum=0
  cutyear=df.loc[df['year']==year]
  for occcode in targetlist:
    cutocc=cutyear.loc[cutyear['occ_code']==occcode]
    employment=cutocc['tot_emp_num'].sum()
    temp1=Abilities.loc[Abilities['SOC Code']==occcode]  
    score_series=temp1.loc[temp1['Element ID']=='1.A.2.a.3']['Data Value']
    try:
      score=float(score_series)
    except:
      continue      
    weightedscore=int(employment*score)
    tempsum+=weightedscore
  FingDexlist.append(tempsum)

In [102]:
FingDexlist

[24584630,
 24114146,
 21950130,
 19897246,
 19706752,
 20027977,
 20523187,
 21382383,
 22214392,
 22923699]

#### Make a list for Routine Cognitive

In [0]:
# Make a list for 10 years, use 'Guiding, Directing and Motivating' in Work Activities, Element id is 4.A.4.b.4
GDMlist=[]
for year in range(2007,2017):
  tempsum=0
  cutyear=df.loc[df['year']==year]
  for occcode in targetlist:
    cutocc=cutyear.loc[cutyear['occ_code']==occcode]
    employment=cutocc['tot_emp_num'].sum()
    temp1=Activities.loc[Activities['occ_code']==occcode]  
    score_series=temp1.loc[temp1['Element ID']=='4.A.4.b.4']['Data Value']
    try:
      score=float(score_series)
    except:
      continue      
    weightedscore=int(employment*score)
    tempsum+=weightedscore
  GDMlist.append(tempsum)

In [109]:
GDMlist

[22618677,
 22206466,
 20155615,
 18257599,
 18135412,
 18468149,
 18987629,
 19778819,
 20597364,
 21233458]

##Conduct 2*2 Autor analysis for Manufature 

In [0]:
!wget "https://www.dropbox.com/s/mzly495b0ns41wi/oes_nat4d_2007-2016.csv?dl=0" -q -O "oes_nat4d_2007-2016.csv" 
allOLS=pd.read_csv("oes_nat4d_2007-2016.csv",low_memory=False)

In [0]:
allOLS

In [0]:
# make a series of Manufactureing 4-digit NAICS code
!wget "https://www.dropbox.com/s/sw4i9fj6lgnlq58/4digit_manu.csv?dl=0" -q -O "4digit_manu.csv" 
manu_naisc=pd.read_csv("4digit_manu.csv",low_memory=False)

In [0]:
# Filter out manufature OLS data
manudf=allOLS.loc[allOLS['naics'].isin(manu_naisc['naisc'])]

In [0]:
# Create occupation list for manufature, and exclude those occ_code end with 00
manu_occ=manudf['occ_code'].unique()
manu_occ_list=[]
for row in manu_occ:
  temp=str(row)
  if temp[5:]!='00':
    new=temp[:2]+temp[3:]
    new=int(new)
    manu_occ_list.append(new)
  else:
    continue

In [0]:
# There are 918 occupations in Manufaturinng
manu_occ_list

In [0]:
manudf

In [0]:
manudf['occ_code']

In [0]:
for temp2 in manudf['occ_code']:
  temp=temp2[:2]+'-'+temp2[2:] 
  manudf[['occ_code']].replace(temp, temp2)

In [0]:
# Replace occ_code in manudf with the correct form of occ_code
for new in manu_occ_list:
  temp2=str(new)
  temp=temp2[:2]+'-'+temp2[2:]  
  manudf[['occ_code']] = manudf[['occ_code']].replace(temp, temp2)



In [240]:
manudf

,year,naics,naics_title,occ_code,occ_title,group,tot_emp,emp_prse,pct_total,pct_rpt,...,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly,occ_group,full_naics
3284,2007,311100,Animal Food Manufacturing,00-0000,Industry Total,NaN,50210,3.3,100.00,100,...,27.68,18010,22770,30810,41090,57580,NaN,NaN,NaN,NaN
3285,2007,311100,Animal Food Manufacturing,11-0000,Management occupations,major,2120,4.4,4.22,30,...,#,43170,57930,78920,111740,#,NaN,NaN,NaN,NaN
3286,2007,311100,Animal Food Manufacturing,111011,Chief executives,NaN,100,11.8,0.20,3,...,#,55350,76500,132360,#,#,NaN,NaN,NaN,NaN
3287,2007,311100,Animal Food Manufacturing,111021,General and operations managers,NaN,940,5.4,1.87,22,...,#,43330,57150,76630,109350,#,NaN,NaN,NaN,NaN
3288,2007,311100,Animal Food Manufacturing,112022,Sales managers,NaN,110,10.2,0.22,3,...,#,51520,73490,96040,126240,#,NaN,NaN,NaN,NaN
3289,2007,311100,Animal Food Manufacturing,113031,Financial managers,NaN,100,12.8,0.20,3,...,#,54420,72230,93280,122270,#,NaN,NaN,NaN,NaN
3290,2007,311100,Animal Food Manufacturing,113051,Industrial production managers,NaN,550,7.4,1.10,13,...,67.52,41760,56350,74140,96630,140450,NaN,NaN,NaN,NaN
3291,2007,311100,Animal Food Manufacturing,113061,Purchasing managers,NaN,30,13.1,0.06,1,...,63.61,37270,52380,88430,114960,132300,NaN,NaN,NaN,NaN
3292,2007,311100,Animal Food Manufacturing,113071,"Transportation, storage, and distribution mana...",NaN,110,10.9,0.22,3,...,56.87,37170,51660,66430,87520,118280,NaN,NaN,NaN,NaN
3293,2007,311100,Animal Food Manufacturing,119199,"Managers, all other",NaN,40,13.8,0.08,1,...,#,56190,73990,101380,#,#,NaN,NaN,NaN,NaN


In [233]:
manudf['occ_code']

3284      00-0000
3285      11-0000
3286      11-1011
3287      11-1021
3288      11-2022
3289      11-3031
3290      11-3051
3291      11-3061
3292      11-3071
3293      11-9199
3294      13-0000
3295      13-1021
3296      13-1023
3297      13-1072
3298      13-1081
3299      13-1199
3300      13-2011
3301      15-0000
3302      15-1021
3303      15-1041
3304      15-1071
3305      17-0000
3306      17-2041
3307      17-2112
3308      17-2141
3309      17-3026
3310      19-0000
3311      19-1012
3312      19-2031
3313      19-3021
           ...   
613993    51-9150
613994    51-9151
613995    51-9190
613996    51-9191
613997    51-9192
613998    51-9194
613999    51-9195
614000    51-9196
614001    51-9198
614002    51-9199
614003    53-0000
614004    53-1000
614005    53-1020
614006    53-1021
614007    53-1030
614008    53-1031
614009    53-3000
614010    53-3030
614011    53-3032
614012    53-3033
614013    53-7000
614014    53-7020
614015    53-7021
614016    53-7050
614017    

In [0]:
manudf['occ_code'].replace(temp,temp2)

In [0]:
newmanudf

In [194]:
manudf['occ_code']=newmanudf

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
manudf

In [0]:
# Make a list for 10 years, use 'negotiation score'in Skills, Element id is 2.B.1.d
negotiationlist=[]
totalemployment=[]
for year in range(2007,2017):
  tempsum=0
  denom=0
  cutyear=manudf.loc[manudf['year']==year]
  for occcode in manu_occ_list:
    cutocc=cutyear.loc[cutyear['occ_code']==occcode]
    employment=cutocc['tot_emp'].sum()
    denom+=employment
    temp1=Skills.loc[Skills['SOC Code']==occcode]  
    score_series=temp1.loc[temp1['Element ID']=='2.B.1.d']['Data Value']
    try:
      score=float(score_series)
    except:
      continue      
    weightedscore=int(employment*score)
    tempsum+=weightedscore
  valueofyear=tempsum/denom
  negotiationlist.append(valueofyear)
  totalemployment.append(denom)

In [0]:
cutyear=manudf.loc[manudf['year']==2007]
cutyear

In [164]:
totalemployment

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]